# User interface: the compressed likelihood

This notebook focuses on the final product, and how we expect most (non-experts) users to use our compressed likelihood. 

By compressed likelihood we mean a likelihood that has already marginalized over nuisance (astro) parameters, and that uses a reduced set of parameters to describe the cosmological model. 

The compressed likelihood doesn't know about redshift bins, or band powers, it doesn't know about mean flux, temperature or redshift or reionization.

Summary:
    - Given an input cosmological model, it computes the parameters describing the linear power spectrum (linP).
    - Given a set of linP parameters, it calls a precomputed object with the likelihood for these parameters.

In [1]:
%matplotlib inline
import numpy as np
import os
## Set default plot size, as normally its a bit too small
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 120
mpl.rcParams['figure.dpi'] = 120
import camb_cosmo
import fit_pk_kms
import lya_results

### Specify cosmological models

We can specify the cosmological model by hand or from a GenIC parameter file (see other notebooks in this folder).

In [2]:
# specify cosmology
cosmo = camb_cosmo.get_cosmology(mnu=0.3, ns=0.96, omch2=0.12)
# print relevant information about the cosmology object
camb_cosmo.print_info(cosmo)

H0 = 6.7000E+01, Omega_b h^2 = 2.2000E-02, Omega_b h^2 = 1.2000E-01, Omega_k = 0.0000E+00, Omega_nu h^2 = 3.2257E-03, T_CMB = 2.7255E+00, A_s = 2.1000E-09, n_s = 9.6000E-01, alpha_s = 0.0000E+00


### Compute parameters describing the linear power spectrum around $z_\star=3$, $k_p=0.009$ s/km

In [3]:
z_star=3.0
kp_kms=0.009
linP_params=fit_pk_kms.parameterize_cosmology_kms(cosmo,z_star=z_star,kp_kms=kp_kms)
print(linP_params)

{'f_star': 0.96901088538702829, 'g_star': 0.9687130850033866, 'Delta2_star': 0.31923954378274538, 'n_star': -2.3071240909857753, 'alpha_star': -0.21455144240692489, 'linP_kms': poly1d([ -0.10727572,  -2.30712409,  15.97238552])}


### Evaluate compressed likelihood

This is the likelihood that has already been marginalized over nuisance parameters, and is of course, experiment specific. At this point it has lost all information about nuisance parameters, or data points. 

Here we show the case for the compressed likelihood in Chabanier et al. (2019, Figure 20)

In [4]:
def loglike_Chabanier2019(cosmo):
    z_star=3.0
    kp_kms=0.009
    linP_params=fit_pk_kms.parameterize_cosmology_kms(cosmo,z_star=z_star,kp_kms=kp_kms)
    # dimensionless amplitude of linear power at z_star, kp_kms
    Delta2_star = linP_params['Delta2_star']
    # logarithmic slope of linear power at z_star, kp_kms
    n_star = linP_params['n_star']
    # logarighmic curvature (running) of linear power at z_star, kp_kms
    alpha_star = linP_params['alpha_star']
    # logarithmic growth rate at z_star, around k = 1 h/Mpc
    f_star=linP_params['f_star']
    # logarithmic derivative of Hubble parameter, with respect to EdS, at z_star 
    g_star=linP_params['g_star']
    
    # here we would interpolate a look-up table or similar, for now we call Chabanier et al. (2019, Figure 20)
    delta_chi2=lya_results.gaussian_chi2_Chabanier2019(n_star,Delta2_star)
    
    return -0.5*delta_chi2

In [5]:
loglike_Chabanier2019(cosmo)

-14.912881409979789